# Candlestick Chart Visualizer with Plotly

This notebook demonstrates how to visualize candlestick charts and add interactive lines and dots using Plotly. You can load OHLCV data, plot candlesticks, and annotate the chart with lines and markers.

In [1]:
# Multi-File Chart Visualizer Function
import pandas as pd
import json
import plotly.graph_objects as go
from src.ta.technical_analysis import TechnicalAnalysisProcessor
from src.ta.middlewares.zigzag import zigzag_middleware
from src.ta.middlewares.volume_profile import volume_profile_middleware
from src.ta.middlewares.channels import channels_middleware
from src.ta.middlewares.levels import levels_middleware

def visualize_chart(file_path, useLogScale=True):
    """
    Load and visualize a single chart from the given file path
    """
    # Load data
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    symbol = data['symbolName']
    time_frame = data['interval']
    df = data['candles']
    df = pd.DataFrame(df)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    
    print(f"Processing {len(df)} candles for {symbol} at {time_frame} interval from {file_path}")
    
    # Run TA processor
    processor = TechnicalAnalysisProcessor(df, time_frame, useLogScale)
    processor.register_middleware(zigzag_middleware)
    processor.register_middleware(volume_profile_middleware)
    processor.register_middleware(channels_middleware)
    processor.register_middleware(levels_middleware)
    analysis = processor.run()
    df = processor.df
    
    # Create figure
    fig = go.Figure(data=[
        go.Candlestick(
            x=df.index,
            open=df['open'],
            high=df['high'],
            low=df['low'],
            close=df['close'],
            name='Candlestick',
            showlegend=False
        )
    ])
    
    # Draw zigzag lines if present
    if 'zigzag' in analysis:
        for line in analysis['zigzag'].get('lines', []):
            if isinstance(line, tuple) and len(line) == 3:
                (t1, p1, _), (t2, p2, _), line_type = line
                fig.add_trace(go.Scatter(x=[t1, t2], y=[p1, p2], mode='lines', line=dict(color='blue', width=2), name='ZigZag', showlegend=False))
        for pivot in analysis['zigzag'].get('pivots', []):
            if len(pivot) == 3:
                t, p, typ = pivot
                fig.add_trace(go.Scatter(x=[t], y=[p], mode='markers', marker=dict(color='red', size=10), name=f'Pivot {typ}', showlegend=False))
    
    # Draw channel lines if present
    if 'channels' in analysis:
        for line in analysis['channels'].get('lines', []):
            if isinstance(line, tuple) and len(line) == 3:
                (t1, p1, _), (t2, p2, _), line_type = line
                fig.add_trace(go.Scatter(x=[t1, t2], y=[p1, p2], mode='lines', line=dict(color='orange', width=2, dash='dash'), name=line_type, showlegend=True))
    
    # Draw channel lines if present
    if 'levels' in analysis:
        for line in analysis['levels'].get('lines', []):
            if isinstance(line, tuple) and len(line) == 3:
                (t1, p1, _), (t2, p2, _), line_type = line
                fig.add_trace(go.Scatter(x=[t1, t2], y=[p1, p2], mode='lines', line=dict(color='orange', width=2, dash='dash'), name=line_type, showlegend=True))

    # Draw volume profile as boxes if present
    if 'volume_profile' in analysis and 'vp' in analysis['volume_profile']:
        for vp_item in analysis['volume_profile']['vp']:
            if isinstance(vp_item, tuple) and len(vp_item) == 3:
                bin_tuple, bin_vol, norm_vol = vp_item
                if isinstance(bin_tuple, tuple) and len(bin_tuple) == 2:
                    bin_start, bin_end = bin_tuple
                    x0 = df.index[0]
                    bar_length = int(norm_vol * len(df.index))
                    x1 = df.index[min(bar_length, len(df.index)-1)]
                    fig.add_shape(
                        type='rect',
                        x0=x0,
                        x1=x1,
                        y0=bin_start,
                        y1=bin_end,
                        line=dict(color='gold', width=1, dash='dot'),
                        fillcolor='rgba(255,215,0,0.2)'
                    )
    
    # Draw volume profile POC, VAH, VAL levels if present
    if 'volume_profile_periods' in analysis and 'lines' in analysis['volume_profile_periods']:
        for line in analysis['volume_profile_periods']['lines']:
            if isinstance(line, tuple) and len(line) == 3:
                (t1, p1, _), (t2, p2, _), line_type = line
        
                if 'naked_poc_line' in line_type:
                    color = 'white'
                    name = f'nPOC'
                elif 'poc_line' in line_type:
                    color = 'white'
                    name = f'POC'
                elif 'VAH' in line_type:
                    color = 'green'
                    name = f'VAH'
                elif 'VAL' in line_type:
                    color = 'red' 
                    name = f'VAL'
                else:
                    color = 'deepskyblue'
                    name = 'Volume Level'
                
                fig.add_trace(go.Scatter(x=[t1, t2], y=[p1, p2], mode='lines', line=dict(color=color, width=1, dash='dash'), name=name, showlegend=True))

    
    # Update layout
    fig.update_layout(
        title=f'{symbol} - {time_frame} - TA Chart Visualizer', 
        xaxis_title='Time', 
        yaxis_title='Price', 
        template='plotly_dark', 
        yaxis_type='log' if useLogScale else 'linear',
        height=800,
        xaxis=dict(fixedrange=False),
        yaxis=dict(fixedrange=False),
        showlegend=False,  # Remove legend
        xaxis_rangeslider_visible=False  # Remove range slider at bottom
    )
    
    fig.show()
    return fig

print("Multi-file visualizer function created!")

Multi-file visualizer function created!


In [3]:
# Option 1: Visualize multiple files in sequence
# Define the list of files you want to visualize
file_paths = [
    #'./data/BTCUSDT-1h.json',
    './data/BTCUSDT-M.json', 
    './data/BTCUSDT-W.json',
    './data/BTCUSDT-D.json'
]

# Visualize each file
for file_path in file_paths:
    try:
        print(f"\n{'='*50}")
        print(f"Visualizing: {file_path}")
        print('='*50)
        visualize_chart(file_path, useLogScale=True)
    except Exception as e:
        print(f"Error processing {file_path}: {e}")


Visualizing: ./data/BTCUSDT-M.json
Processing 68 candles for BTCUSDT at M interval from ./data/BTCUSDT-M.json
Volume Profile: Generated 18 volume profile ranges



Visualizing: ./data/BTCUSDT-W.json
Processing 289 candles for BTCUSDT at W interval from ./data/BTCUSDT-W.json
Volume Profile: Generated 201 volume profile ranges



Visualizing: ./data/BTCUSDT-D.json
Processing 423 candles for BTCUSDT at D interval from ./data/BTCUSDT-D.json
  Period 2024-W30: SKIPPED - insufficient data
Volume Profile: Generated 183 volume profile ranges
